In [ ]:
#from utilities import getLogger

#logger = getLogger(__name__)

import pandas as pd
from datetime import datetime
from collections import defaultdict
from module.tools import (
    generic_error_handling,
    generic_output_handling,
    MetricName,
    is_valid_date,
    is_valid_int,
    is_valid_alert_level,
    log_missing_column,
    log_multi_metric_id,
)

In [ ]:
def HO_PRMD_DOIA001(filename):
    this_metric = MetricName.HO_PRMD_DOIA001
    error_log = {"Column": [], "Row No.": [], "Description": []}

    df = pd.read_excel(filename, skiprows=[])
    df = df.rename(columns=str.lower)
    df = df.filter(["metric id", "date due", "date reported"])

    has_no_missing_columns = True
    try:
        df["date due"] = pd.to_datetime(
            df["date due"], errors="coerce", format="%m/%d/%Y"
        )
    except KeyError:
        has_no_missing_columns = False
        error_log = log_missing_column(error_log, "date due")

    try:
        df["date reported"] = pd.to_datetime(
            df["date reported"], errors="coerce", format="%m/%d/%Y"
        )
    except KeyError:
        has_no_missing_columns = False
        error_log = log_missing_column(error_log, "date reported")

    try:
        df["metric id"] = pd.to_numeric(
            df["metric id"], errors="coerce", downcast="integer"
        )
    except KeyError:
        has_no_missing_columns = False
        error_log = log_missing_column(error_log, "metric id")

    logger.debug(df)

    try:
        num_of_metric_id = len(pd.unique(df["metric id"]))
        unique_metric_id = num_of_metric_id == 1
        if unique_metric_id is False:
            error_log = log_multi_metric_id(error_log, "metric id", num_of_metric_id)
    except KeyError:
        # No Metric ID column handled above
        unique_metric_id = False

    metric_data = defaultdict(list)

    validate_columns = [
        {"name": "metric id", "function": is_valid_int},
        {"name": "date due", "function": is_valid_date},
        {"name": "date reported", "function": is_valid_date},
    ]

    for ndx, data in df.iterrows():
        has_error = False
        for column in validate_columns:
            if column["name"] not in data:
                has_error = True
                continue

            res = column["function"](column["name"], ndx, data)
            if res["success"] is False:
                has_error = True
                error_log["Column"].append(column["name"])
                error_log["Row No."].append(ndx + 1)
                error_log["Description"].append(res["error"])

        if has_error is False and has_no_missing_columns and unique_metric_id:
            metric_value = int(data["date reported"] < data["date due"])
            metric_data[data["metric id"]].append(metric_value)

    if len(error_log["Column"]):
        generic_error_handling(this_metric, filename, error_log)
        return {"message": "Invalid file contents!"}

    data = {"Metric": [], "Input Value": [], "Metric Date": []}
    for k, v in metric_data.items():
        data["Metric"].append(k)
        data["Input Value"].append(sum(v) / len(v) * 100)
        data["Metric Date"].append(datetime.now().strftime("%m/%d/%Y"))

    generic_output_handling(this_metric, filename, data)

    
    return data

In [ ]:
individual notebooks for 
#upload file button and title box

formula

metric id checker
choose columns #columns as buttons auto generate

premade formula dropdown

Generate output
this metric
filename
data